# Understanding Methane Spectra



In [ ]:
import os
import sys
import math
import numpy as np
import pandas as pd
import xarray as xr
from osgeo import gdal
import geopandas as gpd
import earthaccess
import folium.plugins
import rasterio as rio
import rioxarray as rxr

import holoviews as hv
import hvplot
import hvplot.xarray
import hvplot.pandas

sys.path.append('../modules/')
from emit_tools import emit_xarray, ortho_xr
from tutorial_utils import results_to_geopandas

In [ ]:
import earthaccess
sys.path.append('../modules/')

# Data Collections for our search, using a dictionary
concept_id = 'C2748088093-LPCLOUD'
# Define Date Range
date_range = ('2022-08-15','2022-08-16')
results = earthaccess.search_data(
    concept_id=concept_id,
    temporal=date_range,
    count=2000
)
gdf = results_to_geopandas(results)
gdf = gdf.loc[0:2]
gdf

In [ ]:
# url = 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/EMITL1BRAD.001/EMIT_L1B_RAD_001_20220815T042838_2222703_003/EMIT_L1B_RAD_001_20220815T042838_2222703_003.nc'
# earthaccess.login(persist=True)
# fs = earthaccess.get_requests_https_session()
# # Retrieve granule asset ID from URL (to maintain existing naming convention)
# granule_asset_id = url.split('/')[-1]
# # Define Local Filepath
# fp = f'../../data/{granule_asset_id}'
# # Download the Granule Asset if it doesn't exist
# if not os.path.isfile(fp):
#     with fs.get(url,stream=True) as src:
#         with open(fp,'wb') as dst:
#             for chunk in src.iter_content(chunk_size=64*1024*1024):
#                 dst.write(chunk)

In [ ]:
fp = '../../data/EMIT_L1B_RAD_001_20220815T042838_2222703_003.nc'

In [ ]:
rad = emit_xarray(fp, ortho=False)

In [ ]:
enh = rxr.open_rasterio('../../data/EMIT_L2B_CH4ENH_001_20220815T042838_2222703_003.tif').squeeze('band',drop=True)
enh.data[enh.data == -9999] = np.nan

In [ ]:
# rgb = ortho_xr(rad.sel(wavelengths=[625, 525, 460], method='nearest'))
# rgb

In [ ]:
# # Function to adjust gamma across all bands - adjust brightness
# def gamma_adjust(rgb_ds, bright=0.2, white_background=False, var='reflectance'):
#     array = rgb_ds[var].data
#     gamma = math.log(bright)/math.log(np.nanmean(array)) # Create exponent for gamma scaling - can be adjusted by changing 0.2 
#     scaled = np.power(array,gamma).clip(0,1) # Apply scaling and clip to 0-1 range
#     if white_background == True:
#         scaled = np.nan_to_num(scaled, nan = 1) # Assign NA's to 1 so they appear white in plots
#     rgb_ds[var].data = scaled
#     return rgb_ds

In [ ]:
# rgb = gamma_adjust(rgb, white_background=True, var='radiance')

In [ ]:
# map = rgb.hvplot.rgb(x='longitude', y='latitude', bands='wavelengths', geo=True,crs='EPSG:4326', frame_height=500)

In [ ]:
# map = rgb.hvplot.rgb(x='crosstrack', y='downtrack', bands='wavelengths', aspect = 'equal', frame_height=500)

In [ ]:
# Open csv files
points = pd.read_csv('../../data/methane_tutorial/rad_band_ratio_points.csv')
target_spectra = pd.read_csv('../../data/methane_tutorial/emit20220815t042838_o22703_s000_l1b_dua_spec_ch4.csv')

In [ ]:
target_spectra

In [ ]:
points = points.set_index(['ID'])

In [ ]:
# Extract target spectra from dataset
point_ds = rad.sel(crosstrack=(rad.dims['crosstrack']-points.to_xarray().x), downtrack=points.to_xarray().y)

In [ ]:
point_ds = point_ds.drop_vars(['glt_x','glt_y','elev'])

In [ ]:
point_df = point_ds.to_dataframe().join(points['in-plume'],on=['ID'])

In [ ]:
point_df

In [ ]:
# point_df.to_csv('../../data/methane_tutorial/point_df.csv')

In [ ]:
rad.radiance.isel(wavelengths=37).hvplot.image(aspect='equal').opts(invert_yaxis=True)*point_df.hvplot.points(x='crosstrack',y='downtrack', color='in-plume', hover='ID')


In [ ]:
enh.hvplot.image(x='x',y='y',cmap='viridis', geo=True, crs='EPSG:4326')*point_df.hvplot.points(x='lon',y='lat', color='in-plume', cmap='reds', hover_cols='ID', geo=True, crs='EPSG:4326', frame_width=500, frame_height=500)*gdf.hvplot.polygons(geo=True, crs='EPSG:4326', fill_color=None, line_color='red', alpha=0.5)

In [ ]:
# enh.sel(x=53.7749,y=39.4628, method='nearest').x.data

In [ ]:
# def get_ortho_coord(data, coord):
#     diff = np.abs(data - coord)
#     index = diff.argmin()
#     return index

In [ ]:
# rad.glt_x.sel(ortho_y=get_ortho_coord(enh.y.data, 39.4628),ortho_x=get_ortho_coord(enh.x.data,53.7749)).data, rad.glt_y.sel(ortho_y=get_ortho_coord(enh.y.data, 39.4628),ortho_x=get_ortho_coord(enh.x.data,53.7749)).data

In [ ]:
point_df.hvplot(x='wavelengths',y='radiance', by=['ID'], frame_height=400, frame_width=600)

In [ ]:
in_plume = point_df.loc[point_df['in-plume'] == 1].copy()
out_plume = point_df.loc[point_df['in-plume'] == 0].copy()

In [ ]:
in_plume

In [ ]:
out_plume['band_ratio'] = (in_plume.loc[2,'radiance']/out_plume['radiance'])

In [ ]:
ratio_plot = out_plume.hvplot(x='wavelengths',y='band_ratio', by=['ID'], frame_height=200, frame_width=600, xlim=(2150,2450), ylim=(0.85,1.05))

In [ ]:
target_plot = target_spectra.hvplot(x='wavelength',y='value', frame_height=200, frame_width=600, line_color='black', line_width=2, xlim=(2150,2450), ylim=(-1.3,0))

In [ ]:
from bokeh.models import GlyphRenderer, LinearAxis, LinearScale, Range1d

def overlay_hook(plot, element):
    # Adds right y-axis
    p = plot.handles["plot"]
    p.extra_y_scales = {"right": LinearScale()}
    p.extra_y_ranges = {"right": Range1d(-1.3,0)}
    p.add_layout(LinearAxis(y_range_name="right"), "right")

   # find the last line and set it to right
    lines = [p for p in p.renderers if isinstance(p, GlyphRenderer)]
    lines[-1].y_range_name = "right"

(ratio_plot.opts(legend_position='bottom') * target_plot.opts(color="k")).opts(hooks=[overlay_hook]).opts(title='In/Out of Plume Band Ratio and Target Spectrum') 